In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

/mnt/weird_vol/miniconda/envs/downstr/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Engine

## Tau leaping algorithm

Tau leaping modifies the Gillespie methodology, it sacrifices exact simulation in favour of an approximate simulation that is quicker to compute.

1. Initialize - Set initial conditions for the system and set leaping size
2. Calculate event rates - for each event types depending on state of the system
3. Monte-Carlo - for each event type sample number of events occuring within the leap
4. Update - Update system state based on number of events
5. Repeat - Repeat steps 2-4 until some stopping criteria is met


## Poisson distribution

The reactions occur as a poisson process - as a result the number of reactions occuring within a given timeframe will follow a poisson distribution.\
The waiting time between reactions follows an exponential distribution (this is used in the Gillespie simulation).

In [2]:
def pois_random_variable(rate):
    """
    Drawing will be done through the Inverse Transform sampling Method.
    So we want to find a number x for which FX(x) = U; where X is a 
    random variable (Poisson variable in this case) and U is a 
    uniform random variable and FX its distribution function 
    (Poisson distribution in this case). So this means that to find x
    (the goal of this whole function) we need to solve for $x = F^{(-1)}_x(U)$.

    So the steps are:
    - calculate the inverse of the Poisson CDF: $F^{(-1)}_x$
        - the inverse of a CDF can also be interpreted as its quantile function.
    - draw a uniform random probability U (from the interval [0,1], of course)
    - calculate $x = F^{(-1)}_x(U)$

    Note that for for the Poisson distribution, the quantile function is not defined 
    in closed form, so it must be calculated numerically.
    
    Given this, here we use scipy implementation of the ppf (quantile).

    Parameters
    ----------
    rate : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """
    u = np.random.random()
    return stats.poisson.ppf(u, rate)

## Simulation engine

In [4]:
def tau_leaping_simulation(L, T, LT, k_bind, k_dissociate, cycles, steps, leap):
    for i in tqdm(range(cycles)):
        for j in range(steps):
            # Calculate partial reaction rates
            r_bind = k_bind * L[i, j] * T[i, j]
            r_dissociate = k_dissociate * LT[i, j]

            # Calculate number of reactions by type
            n_bind = pois_random_variable(rate=r_bind * leap)
            n_dissociate = pois_random_variable(rate=r_dissociate * leap)

            # Apply limits to prevent negative population
            limit_bind = min(L[i,j], T[i,j])
            limit_dissociate = LT[i,j]
            
            n_bind = min(n_bind, limit_bind)
            n_dissociate = min(n_dissociate, limit_dissociate)
            
            # Update populations
            L[i, j+1] = L[i, j] + n_dissociate - n_bind
            T[i, j+1] = T[i, j] + n_dissociate - n_bind
            LT[i, j+1] = LT[i, j] + n_bind - n_dissociate
                
    return L, T, LT


# Simulate with physiological constants

In [5]:
# Fix random seed for repeatability
np.random.seed(1)

# Fix model parameters
k_bind = 0.0146  # Forward reaction rate
k_dissociate = 35  # Backwards reaction rate

L0 = 10000  # Initial number of ligands
T0 = 7000  # Initial number of receptors
LT0 = 0  # Initial number of ligands bound to receptors

leap = 0.005 # Size of leaping steps

steps = 15000  # Number of reactions per trajectory
cycles = 100  # Number of trajectories iterated over

# Set up holder arrays
t = np.arange(steps + 1) * leap
L = np.zeros((cycles, steps + 1))
T = np.zeros((cycles, steps + 1))
LT = np.zeros((cycles, steps + 1))

# Store initial conditions
L[:, 0] = L0
T[:, 0] = T0
LT[:, 0] = LT0

In [7]:
L, T, LT = tau_leaping_simulation(L=L, T=T, LT=LT, k_bind=k_bind, k_dissociate=k_dissociate, cycles=cycles, steps=steps, leap=leap)

 64%|██████▍   | 64/100 [01:45<00:56,  1.57s/it]